In [1]:
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import spacy
from spacy import displacy
from pypdf import PdfReader
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
import re
import spacy
from spacy import displacy
import os
from aip import AipNlp

In [2]:
CHN_CHAR = "[\u4e00-\u9fa5]"
DIR = "./cache/"
CHN_CHARS = CHN_CHAR + "+"
APP_ID = '40987163'
API_KEY = 'hAxuIu7yuU8M4GqQOKnbPHQG'
SECRET_KEY = 'Yfjnpxy3dbz3T0f3nBTly1Id0ExBXGUt'
nlp = spacy.load("zh_core_web_sm")

In [3]:
notice = pd.read_excel("董秘公告v1.xlsx")
notice.head()

,code,name,title,link,time
0,301082,久盛电气,关于聘任董事会秘书、证券事务代表的公告,http://www.cninfo.com.cn/new/disclosure/detail...,2023-09-22
1,301525,儒竞科技,董事会秘书工作细则,http://www.cninfo.com.cn/new/disclosure/detail...,2023-09-21
2,2219,新里程,关于变更董事会秘书及证券事务代表的公告,http://www.cninfo.com.cn/new/disclosure/detail...,2023-09-19 07:56
3,301498,乖宝宠物,董事会秘书工作细则,http://www.cninfo.com.cn/new/disclosure/detail...,2023-09-19
4,301517,陕西华达,股东大会、董事会、监事会、独立董事、董事会秘书制度的建立健全及运行情况说明,http://www.cninfo.com.cn/new/disclosure/detail...,2023-09-18


In [32]:
driver = webdriver.Chrome()
# set max waiting time
wait = WebDriverWait(driver, 10)
wait.until(EC.number_of_windows_to_be(1))
for i in tqdm(range(len(notice))):
    url = notice.loc[i, "link"]
    driver.get(url)
    # open the pdf page
    wait.until(EC.url_contains("PDF"))
    response = requests.get(driver.current_url)
    with open(f'./cache/{notice.loc[i, "name"]+"_"+notice.loc[i, "title"]}.pdf', 'wb') as f:
        f.write(response.content)
    


100%|██████████| 579/579 [03:51<00:00,  2.50it/s]


In [ ]:
 # playground
driver.current_url


'http://static.cninfo.com.cn/finalpage/2023-09-22/1217916420.PDF'

In [33]:
driver.close()

In [4]:
def generateFakeColumn(l, val):
    assert l == len(notice)
    return [val for _ in range(l)]

def deletePatterns(text, patterns):
    res = text
    for p in patterns:
        res = re.sub(p, "", res)
    return res

In [34]:
class TextMatch:
    def __init__(self, text_body, preprocessPatterns, extractNamePatterns, client):
        self.preprocessPatterns = preprocessPatterns
        self.extractNamePatterns = extractNamePatterns
        self.text_body = text_body
        self.client = client

    def assignMatch(self, notice, i):
        match = re.search("同意" + "[^。；]*" + "[聘兼为][任]*" + "[^。；]*" + "董(事)?(?(1)会秘书|秘)[^。；]*", self.text_body)
        if match:
            # print(10) ####
            name = re.sub("(为)?(?(1)[公]*[司]*董(事)?(?(1)会秘书|秘)[^。；]*|[公]*[司]*董(事)?(?(1)会秘书|秘)[^。；]*)", "", match.group())
            name = re.sub("董(事)?(?(1)会秘书|秘)[^。；]*", "", name)
            if "先生" in name or "女士" in name:
                name = re.findall("(" + CHN_CHARS + ")" + "[先女][生士]", name)[0]
                name = self._helper_MatchName(name)
            else:
                name = self._helper_MatchName(name)
            notice.loc[i, "姓名[入职]"] = name
        else:
            # print(20) ####
            match = re.search("[聘][请任]("+ CHN_CHARS +")[为担任][^。；]*董(事)?(?(1)会秘书|秘)[^。；]*", self.text_body)
            if match:
                # print(30) ####
                name = re.sub("(为)?(?(1)[公]*[司]*董(事)?(?(1)会秘书|秘)[^。；]*|[公]*[司]*董(事)?(?(1)会秘书|秘)[^。；]*)", "", match.group())
                name = self._helper_MatchName(name)
                notice.loc[i, "姓名[入职]"] = name
    
    def _resignMatchReason(self, match_str):
        reason = ""
        reason_raw = re.findall("因[为]?([^。；，]+)", match_str)
        if reason_raw:
            reason += ",".join(reason_raw)
        else:
            reason_raw = re.findall("[由根][于据]([^。；，]+)", match_str)
            if reason_raw:
                reason += ",".join(reason_raw)
            else:
                reason_raw = re.findall("为[了]?([^。；，]+)", match_str)
                if reason_raw:
                    reason += ",".join(reason_raw)
        return reason
    
    def _helper_MatchName(self, string):
        string = deletePatterns(string, self.extractNamePatterns)
        # print(string) ####
        if string:
            res = self.client.lexer(string)
            try:
                for wordObj in res["items"]:
                    if wordObj["pos"] == "nr" or wordObj["ne"] == "PER":
                        return wordObj["item"]
            except:
                ### need to improve accuracy
                doc = nlp(string)
                # print(string) ####
                # print(doc.ents) ####
                if len(doc.ents) > 0:
                    res = str(doc.ents[-1])
                    # for token in doc:
                    #     if token.ent_type_ == "PERSON":
                    #         res = token.text
                    return res
                elif len(string) >= 2 and len(string) <= 4:
                    return string
    
    def _resignMatchName(self, match_str):
        name = None
        # print(match.group()) ####
        name_raw = re.findall("因此([^。；，]+)[先女]?[生士]?[^。；，]*[辞不离]", match_str)
        if name_raw:
            self._helper_MatchName(name_raw[0])
        else:
            name_raw = re.findall("[故]([^。；，]+)[辞不离]", match_str)
            if name_raw:
                name = self._helper_MatchName(name_raw[0])
            if not name:
                # print(1) ####
                name_raw = re.search("[因由根][^。；]+[，]([^。；，]+)[辞不离]", match_str)
                if name_raw:
                    # print(name_raw.group()) ####
                    name_raw = re.findall("([^。；，]+)[辞不离]", name_raw.group())
                    # print(name_raw) ####
                    name = self._helper_MatchName(name_raw[0])
                if not name:
                    # print(2) ####
                    name_raw = re.search("[^。；，]+[因由根][^。；]+[辞不离]", match_str)
                    if name_raw:
                        name_raw = re.sub("[因由根][^。；]+", "", name_raw.group())
                        name = self._helper_MatchName(name_raw)
                    if not name:
                        # print(3) ####
                        name_raw = re.search("[^。；，]+[先女]?[生士]?[^。；，]*[辞不离]", match_str)
                        if name_raw:
                            name_raw = re.sub("[先女]?[生士]?(申)?(?(1)请|[辞不离])", "", name_raw.group())
                            name = self._helper_MatchName(name_raw)
        return name
            
    
    def _resignMatchStay(self, match_str):
        doc = nlp(match_str)
        for j,token in enumerate(doc):
            if token.dep_ == "ROOT":
                if [child for child in token.children][-1].dep_ == "conj" and token.head.dep_ == "ROOT":
                    for child in token.children:
                        if child.dep_ == "neg":
                            return False
        return True

    
    def resignMatch(self, notice, i):
        match = re.search(CHN_CHAR + "*" + "[因由为]" + "[^。；]+[辞不离]" + "[^。；代]+董(事)?(?(1)会秘书|秘)[^。；]*", self.text_body)
        # resign with reason
        if match:
            # print(match.group())
            # match name
            name = self._resignMatchName(match.group())
            if name != None:
                notice.loc[i, "姓名[离职]"] = name
                # match reason
                reason = self._resignMatchReason(match.group())
                notice.loc[i, "离职原因"] = reason

                # match stay
                stay = self._resignMatchStay(match.group())
                notice.loc[i, "留在公司"] = stay
            
        else:
            # resign without reason
            # 过滤代行董秘
            match = re.search(CHN_CHARS + "[辞不离][^。；代]+董(事)?(?(1)会秘书|秘)", self.text_body)
            if match:
                # print(match.group())
                # match name
                name = self._resignMatchName(match.group())
                if name:
                    notice.loc[i, "离职原因"] = "无"
                    notice.loc[i, "姓名[离职]"] = name


In [25]:
preprocessPatterns = ["\n", " ", "[\(（][^）)]*[\)）]", "\\《.*?\\》"]
extractNamePatterns = ["原董事会秘书","董事会", "聘任", "聘请", "指定", "同意", "先生", 
                       "女士", "担任", "兼任", "公司", "申请", "因为", "由于", "副总经理", 
                       "总经理", "存在", "提出", "董秘"]


In [24]:
cols = ["姓名[离职]", "姓名[入职]", "离职原因", "留在公司", "代行"]
for c in cols:
    notice[c] = generateFakeColumn(len(notice), None)

client = AipNlp(APP_ID, API_KEY, SECRET_KEY)

In [15]:
# test_client = AipNlp(APP_ID, API_KEY, SECRET_KEY)
# res = test_client.lexer("邓一新因个人原因，辞")

In [16]:
# res["items"]

[{'uri': '',
  'formal': '',
  'ne': '',
  'item': '邓一新',
  'loc_details': [],
  'basic_words': ['邓', '一', '新'],
  'byte_offset': 0,
  'byte_length': 6,
  'pos': 'nr'},
 {'uri': '',
  'formal': '',
  'ne': '',
  'item': '因',
  'loc_details': [],
  'basic_words': ['因'],
  'byte_offset': 6,
  'byte_length': 2,
  'pos': 'p'},
 {'uri': '',
  'formal': '',
  'ne': '',
  'item': '个人',
  'loc_details': [],
  'basic_words': ['个人'],
  'byte_offset': 8,
  'byte_length': 4,
  'pos': 'n'},
 {'uri': '',
  'formal': '',
  'ne': '',
  'item': '原因',
  'loc_details': [],
  'basic_words': ['原因'],
  'byte_offset': 12,
  'byte_length': 4,
  'pos': 'n'},
 {'uri': '',
  'formal': '',
  'ne': '',
  'item': '，',
  'loc_details': [],
  'basic_words': ['，'],
  'byte_offset': 16,
  'byte_length': 2,
  'pos': 'w'},
 {'uri': '',
  'formal': '',
  'ne': '',
  'item': '辞',
  'loc_details': [],
  'basic_words': ['辞'],
  'byte_offset': 18,
  'byte_length': 2,
  'pos': 'n'}]

In [42]:
for i in tqdm(range(len(notice))):

    reader = PdfReader(f'./cache/{notice.loc[i, "name"]+"_"+notice.loc[i, "title"]}.pdf')
    page = reader.pages[0]
    parts = []
    def visitor_body(text, cm, tm, font_dict, font_size):
        y = tm[5]
        if y > 50 and y < 720:
            parts.append(text)

    page.extract_text(visitor_text=visitor_body)
    text_body = "".join(parts)
    text_body = deletePatterns(text_body, preprocessPatterns)

    tm = TextMatch(preprocessPatterns=preprocessPatterns, extractNamePatterns=extractNamePatterns, text_body=text_body, client=client)
    tm.assignMatch(notice, i)
    tm.resignMatch(notice, i)
    
    match = re.findall(CHN_CHARS + "代[行为][^，。；]*董事会秘书", text_body)
    if len(match) > 0:
        # name = re.findall(CHN_CHARS + "[先女][生士]", match[0])[0][4:-2]
        notice.loc[i, "代行"] = match[-1]

100%|██████████| 579/579 [00:08<00:00, 70.35it/s]


In [44]:

notice.to_excel("Resign3.xlsx", index=False)


In [30]:
for f in os.listdir(DIR):
    os.remove(os.path.join(DIR, f))

In [23]:
# unit test

i = 55
reader = PdfReader(f'./cache/{notice.loc[i, "name"]+"_"+notice.loc[i, "title"]}.pdf')
page = reader.pages[0]
parts = []
def visitor_body(text, cm, tm, font_dict, font_size):
    y = tm[5]
    if y > 50 and y < 720:
        parts.append(text)

page.extract_text(visitor_text=visitor_body)
text_body = "".join(parts)
text_body = deletePatterns(text_body, preprocessPatterns)
tm = TextMatch(preprocessPatterns=preprocessPatterns, extractNamePatterns=extractNamePatterns, text_body=text_body, client=client)
tm.assignMatch(notice, i)
print("姓名:", notice.loc[i, "姓名[入职]"])
print("原因:", notice.loc[i, "离职原因"])

同意聘任史地
姓名: None
原因: 工作调整


In [18]:
tm.text_body

'关于副总经理、董事会秘书辞任的公告本公司及董事会全体成员保证信息披露内容的真实、准确和完整，没有虚假记载、误导性陈述或重大遗漏。经中山联合光电科技股份有限公司。根据相关规定，梁绮丽女士的辞任申请自送达公司董事会之日起生效。经公司董事会研究决定，在公司聘任新的董事会秘书之前，暂由公司财务总监郭耀明先生代为履行董事会秘书职责。公司将按照相关规定尽快完成新任董事会秘书的选聘工作。截至本公告披露日，梁绮丽女士直接持有公司股份27,500股，均为股权激励获授股票，均为有限售条件股份。梁绮丽女士辞去公司副总经理、董事会秘书职务不影响其继续履行作为公司高级管理人员做出的承诺事项。梁绮丽女士所负责的工作已实现平稳交接、过渡，辞去职务后不会对公司生产经营业务、重大事项决策、内部控制有效性等产生不利影响。梁绮丽女士在担任公司副总经理、董事会秘书期间恪尽职守、勤勉尽责，在提升公司治理水平、规范信息披露管理、推动公司资本运作、维护投资者关系等方面发挥了重要作用，有效促进公司健康稳定发展，公司及董事会对梁绮丽女士任职期间为公司做出的贡献表示衷心感谢！特此公告。中山联合光电科技股份有限公司董事会二〇二三年九月五日'

In [170]:
# unit test

abc =  ''' 
深圳文科园林股份有限公司（以下简称“公司”）董事会于近日收到公司
董事会秘书程玉姣女士的书面辞任报告。因个人原因，程玉姣女士申请辞去公
司董事会秘书职务，辞职后将不在公司及公司控股子公司担任任何职务，辞职
报告自本公告披露之日起生效。
'''


abc = re.sub("\\（.*?\\）", "", abc)
abc = re.sub("\n", "", abc)
re.search("[。；，、：][因由][^。；]+[，]([^。；，]+)[先女]?[生士]?[^。，；]*[辞不离]", abc).group()


'。因个人原因，程玉姣女士申请辞去公司董事会秘书职务，辞职后将不'

In [ ]:
abc =  ''' 

经中山联合光电科技股份有限公司（以下简称“公司”)于 2021 年 3 月 17 日
召开的第三届董事会第一次会议审议同意，梁绮丽女士自 2021 年 3 月 17 日起受聘
任担任公司副总经理、董事会秘书，任期至公司第三届董事会任期届满之日止。；
近日，公司董事会收到梁绮丽女士的辞任申请，因公司经营管理工作需要，拟
对其工作岗位进行调整，故梁绮丽女士申请辞去公司副总经理、董事会秘书职务，
辞去前述职务后其仍将在公司任职（非董监高职务）。；

'''

abc = re.sub("\\（.*?\\）", "", abc)
abc = re.sub("\n", "", abc)
re.search(CHN_CHAR + "*" + "[因由为]" + "[^。；]+[辞不离]" + "[^。；]+董事会秘书[^。；]*", abc).group()

if match:
    notice.loc[i, "离职"] = True
    doc = nlp(match[0])
    reasonPtr = 0
    reason = ""
    stay = True
    for j,token in enumerate(doc):
        if token.dep_ == "case":
            reasonPtr = j
        elif token.dep_ == "nmod:prep":
            reason = doc[reasonPtr:j+1].text
            notice.loc[i, "原因"] = reason
        elif token.dep_ == "ROOT":
            for t in token.children:
                if t.dep_ == "nsubj":
                    if t.children:
                        for c in t.children:
                            if c.dep_ == "compound:nn" and c.text != "总监":
                                notice.loc[i, "姓名"] = c.text
                                break
                    else:
                        notice.loc[i, "姓名"] = t.text
                    if notice.loc[i, "姓名"] == '':
                        raise ValueError("fail to extract name:", i)
                    break
            for t in token.children:
                if token.dep_ == "conj" and token.head.dep_ == "ROOT":
                    for child in token.children:
                        if child.dep_ == "neg":
                            stay = False
                            break
                if not stay:
                    break
            break   
    notice.loc[i, "留在公司"] = stay

In [51]:
import spacy
from spacy import displacy

nlp = spacy.load("zh_core_web_sm")
doc = nlp("秘书徐少璞")
print(doc.text)
print(len(doc.ents))
for token in doc:
    print(token.text, token.ent_type_, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])
displacy.render(doc, style="dep")

秘书徐少璞
0
秘书  dep 璞 NOUN []
徐少  dep 璞 NOUN []
璞  ROOT 璞 NOUN [秘书, 徐少]


In [ ]:

nlp = spacy.load("zh_core_web_sm")

刘军

In [12]:
doc = nlp("由于工作调动原因，郭颂华先生申请辞去公司董事会秘书职务，辞职后不在公司担任任何职务。；")
print(doc.text)
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])
displacy.render(doc, style="dep")

由于工作调动原因，郭颂华先生申请辞去公司董事会秘书职务，辞职后不在公司担任任何职务。
由于 case 调动 VERB []
工作 nsubj 调动 VERB []
调动 nmod:prep 申请 VERB [由于, 工作, 原因]
原因 dobj 调动 VERB []
， punct 申请 VERB []
郭颂华 compound:nn 先生 NOUN []
先生 nsubj 申请 VERB [郭颂华]
申请 ROOT 申请 VERB [调动, ，, 先生, 辞去, 担任, 。]
辞去 ccomp 申请 VERB [职务]
公司 compound:nn 职务 NOUN []
董事会 compound:nn 职务 NOUN []
秘书 compound:nn 职务 NOUN []
职务 dobj 辞去 VERB [公司, 董事会, 秘书]
， punct 担任 VERB []
辞职 advcl:loc 担任 VERB [后]
后 case 辞职 VERB []
不 neg 担任 VERB []
在 case 公司 NOUN []
公司 nmod:prep 担任 VERB [在]
担任 conj 申请 VERB [，, 辞职, 不, 公司, 职务]
任何 det 职务 NOUN []
职务 dobj 担任 VERB [任何]
。 punct 申请 VERB []


In [44]:
doc = nlp("苏州恒久光电科技股份有限公司（以下简称“公司”）董事会于近日收到公司董事会秘书洪涛先生提交的书面辞职报告，其因个人原因，申请辞去公司董事会秘书职务，辞职后不再在公司担任任何职务")
print(doc.text)
for token in doc:
    print(token.text, token.dep_, token.pos_, token.head.text, token.head.pos_,
            [child for child in token.children])
displacy.render(doc, style="dep")

苏州恒久光电科技股份有限公司（以下简称“公司”）董事会于近日收到公司董事会秘书洪涛先生提交的书面辞职报告，其因个人原因，申请辞去公司董事会秘书职务，辞职后不再在公司担任任何职务
苏州 compound:nn PROPN 恒久 NOUN []
恒久 compound:nn NOUN 公司 NOUN [苏州]
光电 compound:nn NOUN 公司 NOUN []
科技 compound:nn NOUN 公司 NOUN []
股份 compound:nn NOUN 公司 NOUN []
有限 amod ADJ 公司 NOUN []
公司 ROOT NOUN 公司 NOUN [恒久, 光电, 科技, 股份, 有限, （, 简称]
（ punct PUNCT 公司 NOUN []
以下 advmod ADV 简称 VERB []
简称 dep VERB 公司 NOUN [以下, 公司, ）]
“ punct PUNCT 公司 NOUN []
公司 dobj NOUN 简称 VERB [“, ”]
” punct PUNCT 公司 NOUN []
） punct PUNCT 简称 VERB []
董事会 nsubj NOUN 收到 VERB []
于 case ADP 近日 NOUN []
近日 nmod:prep NOUN 收到 VERB [于]
收到 ROOT VERB 收到 VERB [董事会, 近日, 报告, ，, 原因, ，, 申请, 担任]
公司 compound:nn NOUN 董事会 NOUN []
董事会 compound:nn NOUN 秘书 NOUN [公司]
秘书 compound:nn NOUN 先生 NOUN [董事会]
洪涛 compound:nn PROPN 先生 NOUN []
先生 nsubj NOUN 提交 VERB [秘书, 洪涛]
提交 acl VERB 报告 NOUN [先生, 的]
的 mark PART 提交 VERB []
书面 amod ADJ 报告 NOUN []
辞职 compound:nn NOUN 报告 NOUN []
报告 dobj NOUN 收到 VERB [提交, 书面, 辞职]
， punct PUNCT 收到 VERB []
其因 det DET 原因 NOUN []
个人 compound:nn N

In [45]:

17/579

0.02936096718480138